# 차원 축소 (Dimensionality Reduction)

**학습 목표:**
- 고차원 데이터(수십~수천 개의 특성)가 가지는 문제점인 **'차원의 저주'**를 이해합니다.
- 데이터의 분산을 최대한 보존하며 차원을 축소하는 선형 기법인 **주성분 분석(PCA)**의 원리를 배우고, **누적 설명 분산 비율**을 통해 최적의 주성분 개수를 결정하는 방법을 학습합니다.
- 고차원 공간에서의 데이터 포인트 간 유사성을 저차원 공간에 시각화하는 비선형 기법인 **t-SNE**를 적용하고 PCA와 결과를 비교합니다.
- MNIST 손글씨 숫자 데이터셋을 2차원으로 축소하여 시각화함으로써, 차원 축소의 효과를 직접 확인합니다.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import fetch_openml
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

### (1) 데이터 준비: MNIST 손글씨 숫자
MNIST 데이터는 28x28 픽셀의 손글씨 숫자 이미지로, 각 이미지는 784개의 특성(픽셀 값)으로 구성된 고차원 데이터입니다. 이 데이터를 저차원으로 축소하여 시각화해 보겠습니다.

In [ ]:
# fetch_openml을 통해 MNIST 데이터 로드 (시간이 다소 걸릴 수 있습니다)
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='liac-arff')

# 계산 시간을 줄이기 위해 데이터의 일부만 사용 (예: 10000개)
np.random.seed(42)
random_indices = np.random.choice(mnist.data.shape[0], 10000, replace=False)
X = mnist.data[random_indices]
y = mnist.target[random_indices].astype(int)

print(f"Data shape: {X.shape}")

# 데이터 스케일링
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### (2) 주성분 분석 (PCA)
PCA는 여러 특성 간에 존재하는 상관관계를 이용하여, 이 특성들의 선형 조합으로 표현되는 새로운 축(주성분)을 찾습니다. 첫 번째 주성분은 원본 데이터의 분산을 가장 많이 설명하는 축이며, 두 번째 주성분은 첫 번째 주성분과 직교하면서 다음으로 분산을 많이 설명하는 축입니다.

#### 누적 설명 분산 비율
전체 주성분 중 몇 개를 선택해야 원본 데이터의 정보를 충분히 보존할 수 있을지 결정하는 데 사용됩니다. 보통 95%~99%의 분산을 설명하는 지점까지의 주성분을 선택합니다.

In [ ]:
pca = PCA()
pca.fit(X_scaled)
cumsum = np.cumsum(pca.explained_variance_ratio_)

d = np.argmax(cumsum >= 0.95) + 1 # 95%의 분산을 설명하는 주성분 개수

plt.figure(figsize=(8, 5))
plt.plot(cumsum, linewidth=3)
plt.axis([0, 400, 0, 1])
plt.xlabel("Dimensions")
plt.ylabel("Explained Variance")
plt.plot([d, d], [0, 0.95], "k:")
plt.plot([0, d], [0.95, 0.95], "k:")
plt.title(f'95% explained variance at {d} dimensions')
plt.grid(True)
plt.show()

In [ ]:
# 2차원으로 축소하여 시각화
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

df_pca = pd.DataFrame(X_pca_2d, columns=['PC1', 'PC2'])
df_pca['label'] = y

plt.figure(figsize=(12, 8))
sns.scatterplot(x='PC1', y='PC2', hue='label', data=df_pca, palette=sns.color_palette("hsv", 10), s=50, alpha=0.7)
plt.title('PCA of MNIST dataset (2 Components)')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()

### (3) t-SNE (t-Distributed Stochastic Neighbor Embedding)
t-SNE는 고차원에서의 데이터 포인트 간 지역적 유사성을 저차원에서도 최대한 보존하는 데 초점을 맞춘 비선형 시각화 기법입니다. PCA가 데이터의 전역적인 구조를 보존하려는 것과 달리, t-SNE는 비슷한 데이터들을 가까이 모으는 데 더 효과적이어서 시각화 목적으로 널리 사용됩니다.

In [ ]:
# t-SNE는 계산 비용이 높으므로, PCA로 먼저 50차원 정도로 축소한 후 적용하는 것이 일반적입니다.
pca_50 = PCA(n_components=50)
X_pca_50 = pca_50.fit_transform(X_scaled)

tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_pca_50)

df_tsne = pd.DataFrame(X_tsne, columns=['t-SNE1', 't-SNE2'])
df_tsne['label'] = y

plt.figure(figsize=(12, 8))
sns.scatterplot(x='t-SNE1', y='t-SNE2', hue='label', data=df_tsne, palette=sns.color_palette("hsv", 10), s=50, alpha=0.7)
plt.title('t-SNE of MNIST dataset')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.show()
print("PCA 결과보다 t-SNE 결과에서 각 숫자 클래스가 더 명확하게 군집을 이루는 것을 볼 수 있습니다.")